### Delete documents across partitions

In [ ]:
# Install required libraries
%pip install azure-cosmos
%pip install faker

# import azure.cosmos
# print(azure.cosmos.__version__)

In [1]:
# Initialization and config
from azure.cosmos import CosmosClient, PartitionKey
from configparser import ConfigParser
from faker import Faker

import os
import json
import uuid

# Assuming structure Notebooks/Models
parser = ConfigParser()
parser.read('../NotebookConfig.cfg')

cosmosAccountURI = parser.get('CosmosDB', 'COSMOSDB_ACCOUNT_URI')
cosmosAccountKey = parser.get('CosmosDB', 'COSMOSDB_ACCOUNT_KEY')

databaseName = 'Learn'
containerName = 'NoSQL'
partitionKeypath = '/PartitionKey'
osPath = './OutputFiles/'

Faker.seed(0)
fake = Faker(['en-US'])

if not os.path.exists(osPath):
    os.mkdir(osPath)

In [2]:
client = CosmosClient(cosmosAccountURI, cosmosAccountKey)
db = client.create_database_if_not_exists(databaseName)

pkPath = PartitionKey(path=partitionKeypath)
ctr = db.create_container_if_not_exists(id=containerName, partition_key=pkPath, offer_throughput=400)

In [3]:
# Reference list for documents that should have consistent values across cell/operations.
from collections import OrderedDict
maxRange = 10000
IOTSources = []

os.makedirs(os.path.dirname('./OutputFiles/'), exist_ok=True)
with open('./OutputFiles/' + containerName + '_referenceData.json', 'w') as jsonFile:
    for i in range(maxRange):
        entity = {            
            'Name': fake.bothify('????_############')
            , 'Type': fake.random_element(elements=('Type1', 'Type2', 'Type3', 'Type4', 'Type5', 'Type6'))
        }
        IOTSources.append(entity)

        # Save patients for reference
        json.dump(entity, jsonFile)
        if (i < maxRange):
            jsonFile.write(',\n')

In [5]:
# Insert 1K documents with fixed Partition id
from datetime import datetime, timedelta

RUCharges = []
daysRange = 1
iotRange = 1000

dtBase = datetime(year=2022, month=12, day=1)

for iot in range(iotRange):
    IOTSrc = IOTSources[iot]

    for day in range(daysRange):
        docs = []        
        readings = []

        # *** Produce 10 readings - 1 for each minute
        for m in range(1):
            readings.append(
                {
                    'Dimension': fake.random_element(elements=('D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9'))
                    , 'Metric': fake.random_number(digits=5)
                    , 'Timestamp': (dtBase + timedelta(days=day,minutes=m)).isoformat()  # fake.date_time_this_year().isoformat()
                })

        doc = {
            'id': str(uuid.uuid4())
            , 'PartitionKey': 'D00001006'
            , 'Name': IOTSrc['Name']
            , 'Type': IOTSrc['Type']            
            , 'Entity': IOTSrc
            , 'Location': {
               "type":"Point",
                "coordinates":[ float(fake.numerify(text='40.#####')), float(fake.numerify(text='-74.#####')) ]     
            }
            , 'DateTime': (dtBase + timedelta(days=day)).isoformat()
            , 'Timestamp': (dtBase + timedelta(days=day)).timestamp()
            , 'Readings': readings
            , 'class': fake.random_element(elements=OrderedDict([("A", 0.40), ("B", 0.35), ("C", 0.15), ("D", 0.05), ("E", 0.05)]))
        }

        ctr.create_item(doc)
        RUCharges.append(float(ctr.client_connection.last_response_headers['x-ms-request-charge']))
        # print(ctr.client_connection.last_response_headers['x-ms-request-charge'])

In [ ]:
# Load 100K documents into container
# Random locations based on geospatial coordiates around latitude 40.xxxxx and longitude -74.xxxxx

from datetime import datetime, timedelta

RUCharges = []
daysRange = 1
iotRange = 1000

dtBase = datetime(year=2022, month=12, day=1)

for iot in range(iotRange):
    IOTSrc = IOTSources[iot]

    for day in range(daysRange):
        docs = []        
        readings = []

        # *** Produce 10 readings - 1 for each minute
        for m in range(1):
            readings.append(
                {
                    'Dimension': fake.random_element(elements=('D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9'))
                    , 'Metric': fake.random_number(digits=5)
                    , 'Timestamp': (dtBase + timedelta(days=day,minutes=m)).isoformat()  # fake.date_time_this_year().isoformat()
                })

        doc = {
            'id': str(uuid.uuid4())
            , 'PartitionKey': IOTSrc['Name'] + '_' + IOTSrc['Type'] + '_' + (dtBase + timedelta(days=day)).strftime('%Y_%m_%d')
            , 'Name': IOTSrc['Name']
            , 'Type': IOTSrc['Type']            
            , 'Entity': IOTSrc
            , 'Location': {
               "type":"Point",
                "coordinates":[ float(fake.numerify(text='40.#####')), float(fake.numerify(text='-74.#####')) ]     
            }
            , 'DateTime': (dtBase + timedelta(days=day)).isoformat()
            , 'Timestamp': (dtBase + timedelta(days=day)).timestamp()
            , 'Readings': readings
            , 'class': fake.random_element(elements=OrderedDict([("A", 0.40), ("B", 0.35), ("C", 0.15), ("D", 0.05), ("E", 0.05)]))
        }

        ctr.create_item(doc)
        RUCharges.append(float(ctr.client_connection.last_response_headers['x-ms-request-charge']))
        # print(ctr.client_connection.last_response_headers['x-ms-request-charge'])

In [10]:
for item in ctr.query_items(query="SELECT c.id, c.PartitionKey FROM c WHERE c.PartitionKey != 'D00001006'" 
    , enable_cross_partition_query=True):
    ctr.delete_item(item=item['id'], partition_key=item['PartitionKey'])
    print(item)


Gisi_660487647593_Type1_2022_12_01
NFvp_387784080160_Type6_2022_12_01
GFgt_513933287115_Type5_2022_12_01
MZyu_418583989471_Type5_2022_12_01
jcfS_342320947112_Type5_2022_12_01
QPSY_684833969477_Type3_2022_12_01
ToxY_159179533041_Type2_2022_12_01
hOmM_560123098910_Type5_2022_12_01
nUZd_161510903217_Type6_2022_12_01
ldGD_086914131456_Type1_2022_12_01
kvHq_916345792302_Type1_2022_12_01
jJSa_972076984564_Type4_2022_12_01
LONi_150842375945_Type6_2022_12_01
yTRK_466109352337_Type4_2022_12_01
FJMW_069602714278_Type1_2022_12_01
zRAu_075470638120_Type1_2022_12_01
SlgE_300891319344_Type4_2022_12_01
bccn_104714285124_Type6_2022_12_01
lnyL_485590977658_Type3_2022_12_01
rkjL_022455515900_Type3_2022_12_01
Uetz_568241730428_Type3_2022_12_01
SFBc_461187755171_Type3_2022_12_01
agzJ_522961113306_Type5_2022_12_01
Bmwh_477936153492_Type1_2022_12_01
jgmD_087317643039_Type4_2022_12_01
RuFF_582197296687_Type6_2022_12_01
MYjy_389305550824_Type5_2022_12_01
WCvk_471180132040_Type2_2022_12_01
VBWn_758688091891_Ty